In [33]:
import numpy as np
import numba as nb
from numba.typed import List
import pandas as pd
import itertools
from operator import itemgetter
import gc

from scipy.stats import norm, lognorm, truncnorm
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter, deque
from sys import getsizeof
from sedgen.initialization import SedGen
from sedgen import initialization as ini
from sedgen import general as gen
from sedgen import preprocessing
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load jupyter extension to reload packages before executing user code.
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all packages (except those excluded by %aimport) every time 
# before executing the Python code typed.
%autoreload 2

minerals = ["Q", "P", "K", "B", "O", "A"]
modal_mineralogy = np.array([0.30591989, 0.38159713, 0.26209888, 0.0188256 , 0.00799247, 0.02356603])
csds_CA_NS_means = np.array([0.309, 0.330, 0.244, 0.223, 0.120, 0.122])
csds_CA_NS_stds = np.array([0.823, 0.683, 0.817, 0.819, 0.554, 0.782])
par_rock = 1e9

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
def optimise_learnrate(minerals, par_rock_volume, modal_mineralogy, means, stds):
    learnrate = 100
    counter = 0
    while counter != 10:
        model = SedGen(minerals, par_rock_volume, modal_mineralogy, means, stds, learning_rate = learnrate, discretization_init=False, timed = True)
        modelled_volume = np.sum(model.simulated_volume)
        print(par_rock_volume)
        print(modelled_volume)
        if modelled_volume < ((par_rock_volume*0.0001)+par_rock_volume) and modelled_volume> (par_rock_volume-(par_rock_volume*0.0001)): 
            break
        else:
            learnrate *= 10
        print(counter, learnrate)
        counter += 1
    pluton_learnrate = learnrate * par_rock_volume
    return (pluton_learnrate)
    
            

In [35]:
optimise_learnrate(minerals, par_rock, modal_mineralogy, csds_CA_NS_means, csds_CA_NS_stds)

---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A| Done in 24.2307 seconds
Initializing interfaces... |Q|P|K|B|O|A|[0 0 0 ... 0 0 0]
[2 0 1 ... 1 0 0]
 Done in 64.2205 seconds
Counting interfaces...  Done in 0.7182 seconds
Correcting interface arrays for consistency...
too few Q -7
too much P 1
too much K 1
too much B 2
too much O 1
too much A 2
Initializing crystal size array... |Q|P|K|B|O|A| Done in 4.4068 seconds
Initializing inter-crystal breakage probability arrays...
Initializing model evolution arrays...

---SedGen model initialization finished succesfully---
1000000000.0
5896667275.989795
0 1000
---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A| Done in 4.8614 seconds
Initializing interfaces... |Q|P|K|B|O|A|[0 0 0 ... 0 0 0]
[1 2 3 ... 0 0 0]
 Done in 12

10000000000000.0